In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import string
import seaborn as sns
import plotly.express as ex
import re

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [3]:
train.head(6)

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0
5,6,On maximizing the fundamental frequency of the...,Let $\Omega \subset \mathbb{R}^n$ be a bound...,0,0,1,0,0,0


In [4]:
train.iloc[5]['ABSTRACT']

'  Let $\\Omega \\subset \\mathbb{R}^n$ be a bounded domain satisfying a\nHayman-type asymmetry condition, and let $ D $ be an arbitrary bounded domain\nreferred to as "obstacle". We are interested in the behaviour of the first\nDirichlet eigenvalue $ \\lambda_1(\\Omega \\setminus (x+D)) $. First, we prove an\nupper bound on $ \\lambda_1(\\Omega \\setminus (x+D)) $ in terms of the distance\nof the set $ x+D $ to the set of maximum points $ x_0 $ of the first Dirichlet\nground state $ \\phi_{\\lambda_1} > 0 $ of $ \\Omega $. In short, a direct\ncorollary is that if \\begin{equation} \\mu_\\Omega := \\max_{x}\\lambda_1(\\Omega\n\\setminus (x+D)) \\end{equation} is large enough in terms of $ \\lambda_1(\\Omega)\n$, then all maximizer sets $ x+D $ of $ \\mu_\\Omega $ are close to each maximum\npoint $ x_0 $ of $ \\phi_{\\lambda_1} $.\nSecond, we discuss the distribution of $ \\phi_{\\lambda_1(\\Omega)} $ and the\npossibility to inscribe wavelength balls at a given point in $ \\Omega $.\nFi

In [5]:
def clean(x):
    stop=set(stopwords.words('english'))
    s=[]
    for w in x.split():
        if w.lower() not in stop:
            s.append(w)
    s=' '.join(s)
    url=re.compile(r'https?://\S+|www\.\S+')
    k=url.sub(r'',s)
    html=re.compile(r'<.*?>')
    k=html.sub(r'',k)
    k=re.sub('\n',' ',k)
    table=str.maketrans('','',string.punctuation)
    k=k.translate(table)
    return k.lower()

In [6]:
clean(train.iloc[5]['ABSTRACT'])

'let omega subset mathbbrn bounded domain satisfying haymantype asymmetry condition let   arbitrary bounded domain referred obstacle interested behaviour first dirichlet eigenvalue  lambda1omega setminus xd  first prove upper bound  lambda1omega setminus xd  terms distance set  xd  set maximum points  x0  first dirichlet ground state  philambda1  0   omega  short direct corollary beginequation muomega  maxxlambda1omega setminus xd endequation large enough terms  lambda1omega  maximizer sets  xd   muomega  close maximum point  x0   philambda1  second discuss distribution  philambda1omega  possibility inscribe wavelength balls given point  omega  finally specify observations convex obstacles   show  muomega  sufficiently large respect  lambda1omega  maximizers  xd   muomega  contain maximum points  x0   philambda1omega '

In [8]:
train['ABSTRACT']=train['ABSTRACT'].apply(clean)
test['ABSTRACT']=test['ABSTRACT'].apply(clean)

In [9]:
train

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,predictive models allow subjectspecific infere...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,rotation invariance translation invariance gre...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,introduce develop notion spherical polyharmoni...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,stochastic landaulifshitzgilbert llg equation ...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,fouriertransform infrared ftir spectra samples...,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
20967,20968,Contemporary machine learning: a guide for pra...,machine learning finding increasingly broad ap...,1,1,0,0,0,0
20968,20969,Uniform diamond coatings on WC-Co hard alloy c...,polycrystalline diamond coatings grown cemente...,0,1,0,0,0,0
20969,20970,Analysing Soccer Games with Clustering and Con...,present new approach identifying situations be...,1,0,0,0,0,0
20970,20971,On the Efficient Simulation of the Left-Tail o...,sum lognormal variates encountered many challe...,0,0,1,1,0,0


In [10]:
from sklearn.svm import LinearSVC

In [11]:
from sklearn.linear_model import LogisticRegression

In [12]:
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.multiclass import OneVsRestClassifier

In [13]:
from sklearn.model_selection import train_test_split,KFold,GridSearchCV

In [18]:
X=train[['ABSTRACT']]
y=train[cats]

In [19]:
X

,ABSTRACT
0,predictive models allow subjectspecific infere...
1,rotation invariance translation invariance gre...
2,introduce develop notion spherical polyharmoni...
3,stochastic landaulifshitzgilbert llg equation ...
4,fouriertransform infrared ftir spectra samples...
...,...
20967,machine learning finding increasingly broad ap...
20968,polycrystalline diamond coatings grown cemente...
20969,present new approach identifying situations be...
20970,sum lognormal variates encountered many challe...


In [20]:
y


,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,0,0,1,0,0,0
3,0,0,1,0,0,0
4,1,0,0,1,0,0
...,...,...,...,...,...,...
20967,1,1,0,0,0,0
20968,0,1,0,0,0,0
20969,1,0,0,0,0,0
20970,0,0,1,1,0,0


In [34]:
params={'penalty':['l1','l2'],'C': [0.001,0.1,1,2,5,10,20,100],'tol':[0.001,0.01,0.0001,0.1],'class_weight':[None,'balanced']}
model=OneVsRestClassifier(LinearSVC(),n_jobs=1)
svc=LogisticRegression()
grid=GridSearchCV(svc,params,verbose=2)
tf=TfidfVectorizer()
X_train1=tf.fit_transform(X['ABSTRACT'])
X_test=tf.transform(test['ABSTRACT'])
for category in cats:
    print('... Processing {}'.format(category))
    grid.fit(X_train1, y[category])
    prediction = grid.predict(X_test)
    sub[category]=prediction

, tol=0.001 ..................
[CV] ... C=20, class_weight=None, penalty=l2, tol=0.001, total=   2.0s
[CV] C=20, class_weight=None, penalty=l2, tol=0.001 ..................
[CV] ... C=20, class_weight=None, penalty=l2, tol=0.001, total=   1.7s
[CV] C=20, class_weight=None, penalty=l2, tol=0.001 ..................
[CV] ... C=20, class_weight=None, penalty=l2, tol=0.001, total=   2.0s
[CV] C=20, class_weight=None, penalty=l2, tol=0.001 ..................
[CV] ... C=20, class_weight=None, penalty=l2, tol=0.001, total=   1.7s
[CV] C=20, class_weight=None, penalty=l2, tol=0.001 ..................
[CV] ... C=20, class_weight=None, penalty=l2, tol=0.001, total=   2.0s
[CV] C=20, class_weight=None, penalty=l2, tol=0.01 ...................
[CV] .... C=20, class_weight=None, penalty=l2, tol=0.01, total=   1.5s
[CV] C=20, class_weight=None, penalty=l2, tol=0.01 ...................
[CV] .... C=20, class_weight=None, penalty=l2, tol=0.01, total=   1.5s
[CV] C=20, class_weight=None, penalty=l2, tol=

In [31]:
params={'penalty':['l1','l2'],'C': [0.001,0.1,1,2,5,10,20,100,1000],'tol':[0.001,0.01,0.0001,0.1],'class_weight':[None,'balanced']}
model=OneVsRestClassifier(LogisticRegression(C=2,tol=0.001,class_weight='balanced',penalty='l2'),n_jobs=1)
tf=TfidfVectorizer()
X_train1=tf.fit_transform(X['ABSTRACT'])
X_test=tf.transform(test['ABSTRACT'])
for category in cats:
    print('... Processing {}'.format(category))
    model.fit(X_train1, y[category])
    prediction = model.predict(X_test)
    sub[category]=prediction


... Processing Computer Science
... Processing Physics
... Processing Mathematics
... Processing Statistics
... Processing Quantitative Biology
... Processing Quantitative Finance


train test split

In [22]:
grid.best_params_

{'C': 2, 'class_weight': 'balanced', 'penalty': 'l2', 'tol': 0.001}

In [82]:
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.3)

In [87]:
tf=TfidfVectorizer()
X_train1=tf.fit_transform(X_train['ABSTRACT'])
X_val1=tf.transform(X_val['ABSTRACT'])
X_test=tf.transform(test['ABSTRACT'])

In [93]:
X_train1

<14680x73312 sparse matrix of type '<class 'numpy.float64'>'
	with 1019452 stored elements in Compressed Sparse Row format>

In [17]:
cats=['Computer Science', 'Physics', 'Mathematics',
       'Statistics', 'Quantitative Biology', 'Quantitative Finance']

In [21]:
sub=pd.DataFrame(columns=['ID','Computer Science', 'Physics', 'Mathematics',
       'Statistics', 'Quantitative Biology', 'Quantitative Finance'])

In [22]:
sub['ID']=test['ID'].values

In [36]:
sub

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,1,0,0
1,20974,0,1,0,0,0,0
2,20975,1,0,0,0,0,0
3,20976,0,1,0,0,0,0
4,20977,1,0,0,0,0,0
...,...,...,...,...,...,...,...
8984,29957,1,0,0,0,0,0
8985,29958,1,0,1,0,0,0
8986,29959,1,0,0,1,1,0
8987,29960,1,0,0,1,0,0


In [35]:
sub.to_csv('pred9.csv',index=False)